### 데이터 수집

1) 한국교육학술정보원(KERIS)의 학술연구정보서비스(RISS) 사이트 이용
    - 1,000개의 해외 학술 문서 제목을 다운로드해 사용
    - 링크: (http://www.riss.kr/index.do)
2) 통합검색 결과 페이지에서 [해외학술논문] 메뉴를 클릭
    - ‘Big data’로 검색한 결과
        - ‘해외학술논문’ 선택 → 17,597 건의 결과 확인(2024.10.30일 기준)
3) ‘작성언어’를 [영어]로 선택하고 ‘실행’ 버튼을 클릭
4) 검색 결과 출력 개수 변경하기
    - ‘100개씩 출력’ → 조회
5) 조회된 검색 결과를 선택하고 내보내기 버튼 선택
6) [Excel저장]을 선택 → 버튼을 클릭한 뒤 파일을 
7) 다음 페이지로 이동하여 이전 과정을 반복
8) 다운로드한 총 10개의 엑셀 파일 확인

In [ ]:
!pip install matplotlib

In [ ]:
!pip install wordcloud

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import glob
import re
from functools import reduce
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud


In [ ]:
all_files = glob.glob('dataSet/exportExcelData_*.xls')
all_files

In [ ]:
!pip install xlrd

In [ ]:
all_files_data = [] #저장할 리스트
for file in all_files: 
    data_frame = pd.read_excel(file) 
    all_files_data.append(data_frame) 
all_files_data[0] #작업 내용 확인

In [ ]:
all_files_data_concat = pd.concat(all_files_data, axis = 0, ignore_index = True) 
all_files_data_concat #출력하여 내용 확인

In [ ]:
all_files_data_concat.to_csv('dataSet/riss_bigdata.csv', encoding = 'utf-8', index = False)

In [ ]:
all_title = all_files_data_concat['제목'] 
all_title #출력하여 내용 확인

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

In [ ]:
stopWords = set(stopwords.words("english"))
lemma = WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
words = [] 
for title in all_title: 
    EnWords = re.sub(r"[^a-zA-Z]+", " ", str(title)) 
    EnWordsToken = word_tokenize(EnWords.lower()) 
    EnWordsTokenStop = [w for w in EnWordsToken if w not in stopWords] 
    EnWordsTokenStopLemma = [lemma.lemmatize(w) for w in EnWordsTokenStop] 
    words.append(EnWordsTokenStopLemma)
print(words)

In [ ]:
words2 = list(reduce(lambda x, y: x+y, words))
print(words2)

In [ ]:
count = Counter(words2)
count

In [ ]:
word_count = dict()
for tag, counts in count.most_common(50):
    if(len(str(tag))>1):
        word_count[tag] = counts
        print("%s : %d" % (tag, counts))

In [ ]:
sorted_Keys = sorted(word_count, key = word_count.get, reverse = True)
sorted_Values = sorted(word_count.values(), reverse = True)
plt.bar(range(len(word_count)), sorted_Values, align = 'center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation = 'vertical')
plt.show()

In [ ]:
del word_count['big']
del word_count['data']

sorted_Keys = sorted(word_count, key = word_count.get, reverse = True)
sorted_Values = sorted(word_count.values(), reverse = True)
plt.bar(range(len(word_count)), sorted_Values, align = 'center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation = 'vertical')
plt.show()

In [ ]:
all_files_data_concat['doc_count'] = 0 
summary_year = all_files_data_concat.groupby('출판일', as_index = False)['doc_count'].count() 
summary_year

In [ ]:
plt.figure(figsize = (12, 5))
plt.xlabel("year")
plt.ylabel("doc-count")
plt.grid(True)
plt.plot(range(len(summary_year)), summary_year['doc_count'])
plt.xticks(range(len(summary_year)), [text for text in summary_year['출판일']])
plt.show()

In [ ]:
stopwords = set(STOPWORDS)
wc = WordCloud(background_color = 'ivory', stopwords = stopwords, width = 1200, height = 
800)
cloud = wc.generate_from_frequencies(word_count)
plt.figure(figsize = (8,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [ ]:
cloud.to_file("dataSet/riss_bigdata_wordCloud.jpg")